In [83]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE, ADASYN

In [84]:
data = pd.read_csv('Bolster_Data_Robot_Input.csv')

In [85]:
#pca
pca1 = PCA(n_components=1)
pc_d_l = principalComponents = pca1.fit_transform(data[['CAR_AGE IN DAYS','LIFE TO DATE MILEAGE']])
data['PCA_d_l']=pc_d_l
#data = data.drop(['Mileage_since_last _failure','weeks_in_service'],axis = 1)

In [86]:
#pca
pca1 = PCA(n_components=1)
pc_w_m = principalComponents = pca1.fit_transform(data[['Mileage since Last Replacement','COMPONENT_AGE IN DAYS']])
data['PCA_w_m']=pc_w_m
#data = data.drop(['Mileage_since_last _failure','weeks_in_service'],axis = 1)

In [87]:
data.head()

,CAR NUMBER,LOCATION,CURRENT_CONDITION_CODE,SHOP_INITIAL,Last Replacement Date,CUT OFF DATE,BOLSTER_TYPE,Mileage since Last Replacement,Car_Class,Equipment_Feature_Desc,...,RR_ST,RR_UP,AVG_Loaded_Trip_Duration_Days,AVG_Loaded_Distance_Miles,AVG_Empty_Trip_Duration_Days,AVG_Empty_Distance_Miles,COMPONENT_AGE IN DAYS,CAR_AGE IN DAYS,PCA_d_l,PCA_w_m
0,83500,A,3,WRRX,8/21/2017,2/28/2019,0,34479,GSH64,64 ft 100T,...,0,12,20.329629,1060.389629,14.625925,581.204074,556,11928,224302.094298,-426575.521036
1,83500,B,3,WRRX,8/21/2017,2/28/2019,0,34479,GSH64,64 ft 100T,...,0,12,20.329629,1060.389629,14.625925,581.204074,556,11928,224302.094298,-426575.521036
2,83503,A,1,0,7/8/1986,2/28/2019,0,798885,GSH64,64 ft 100T,...,0,19,30.188571,1083.472571,16.408571,802.380571,11923,11923,222448.123326,337911.349270
3,83503,B,1,0,7/8/1986,2/28/2019,0,798885,GSH64,64 ft 100T,...,0,19,30.188571,1083.472571,16.408571,802.380571,11923,11923,222448.123326,337911.349270
4,83505,A,1,0,7/8/1986,2/28/2019,0,848482,GSH64,64 ft 100T,...,0,28,17.542857,1087.278367,17.858333,861.054375,11923,11923,272043.155172,387504.906273


In [88]:
data['CURRENT_CONDITION_CODE'] = pd.factorize(data.CURRENT_CONDITION_CODE)[0]
data['SHOP_INITIAL'] = pd.factorize(data.SHOP_INITIAL)[0]
data['BOLSTER_TYPE'] = pd.factorize(data.BOLSTER_TYPE)[0]
data['Equipment_Feature_Desc'] = pd.factorize(data.Equipment_Feature_Desc)[0]
data['Builder_Name'] = pd.factorize(data.Builder_Name)[0]
data['Car_Class'] = pd.factorize(data.Car_Class)[0]

In [89]:
data.head()

,CAR NUMBER,LOCATION,CURRENT_CONDITION_CODE,SHOP_INITIAL,Last Replacement Date,CUT OFF DATE,BOLSTER_TYPE,Mileage since Last Replacement,Car_Class,Equipment_Feature_Desc,...,RR_ST,RR_UP,AVG_Loaded_Trip_Duration_Days,AVG_Loaded_Distance_Miles,AVG_Empty_Trip_Duration_Days,AVG_Empty_Distance_Miles,COMPONENT_AGE IN DAYS,CAR_AGE IN DAYS,PCA_d_l,PCA_w_m
0,83500,A,0,0,8/21/2017,2/28/2019,0,34479,0,0,...,0,12,20.329629,1060.389629,14.625925,581.204074,556,11928,224302.094298,-426575.521036
1,83500,B,0,0,8/21/2017,2/28/2019,0,34479,0,0,...,0,12,20.329629,1060.389629,14.625925,581.204074,556,11928,224302.094298,-426575.521036
2,83503,A,1,1,7/8/1986,2/28/2019,0,798885,0,0,...,0,19,30.188571,1083.472571,16.408571,802.380571,11923,11923,222448.123326,337911.349270
3,83503,B,1,1,7/8/1986,2/28/2019,0,798885,0,0,...,0,19,30.188571,1083.472571,16.408571,802.380571,11923,11923,222448.123326,337911.349270
4,83505,A,1,1,7/8/1986,2/28/2019,0,848482,0,0,...,0,28,17.542857,1087.278367,17.858333,861.054375,11923,11923,272043.155172,387504.906273


In [79]:
best_vals_reg=[]
best_vals_reg_pca = []
best_vals_reg_oversampling = []
best_vals_reg_oversampling_pca = []
for j in range(10):
    data_unique_car_number = pd.DataFrame(data['CAR NUMBER'].unique())
    data_unique_car_number.columns=['CAR NUMBER']
    
    data_random=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    data_random.columns=['RandomNum']
    
    data_unique_car_number['RandomNum']=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    
    data_merged =pd.merge(data,data_unique_car_number,on='CAR NUMBER',how='left')

    data_new = data_merged.drop(['CAR NUMBER','LOCATION', 'Last Replacement Date','CUT OFF DATE','BUILT DATE'],axis =1)
    
    #train and test  
    data_train_temp = data_new[data_new['RandomNum'] <= 0.75]
    data_train = data_train_temp.drop(['RandomNum'],axis =1)
    
    data_test_temp = data_new[data_new['RandomNum'] > 0.75]
    data_test = data_test_temp.drop(['RandomNum'],axis =1)
    
    y_train = data_train['TARGET']
    x_train = data_train.drop(['TARGET'],axis =1)
    y_test = data_test['TARGET']
    x_test = data_test.drop(['TARGET'],axis =1)
    
    
    ada = ADASYN(random_state=42)
    
    y_train = data_train['TARGET']
    x_train_pca = data_train.drop(['TARGET'],axis =1)
    x_train = data_train.drop(['TARGET','PCA_d_l','PCA_w_m'],axis =1)
    x_train_oversampling, y_train_oversampling = ada.fit_resample(x_train, y_train)
    x_train_oversampling_pca, y_train_oversampling_pca = ada.fit_resample(x_train_pca, y_train)

    
    y_test = data_test['TARGET']
    x_test = data_test.drop(['TARGET','PCA_d_l','PCA_w_m'],axis =1)
    x_test_pca = data_test.drop(['TARGET'],axis =1)
    
    
    clf_pca = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=3000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20).fit(x_train_pca,y_train)
    
    clf = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=3000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20).fit(x_train,y_train)
    
    clf_oversampling = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=3000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20).fit(x_train_oversampling,y_train_oversampling)
    
    clf_oversampling_pca = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=3000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20).fit(x_train_oversampling_pca,y_train_oversampling_pca)
    
    
    vals=[]
    for i in np.linspace(0.0005,0.01,100):
        predicted_probas = clf.predict_proba(x_test)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals.append(metrics.auc(fpr, tpr))
    print('{0}-{1}'.format(max(vals),np.linspace(0.0005,0.01,100)[vals.index(max(vals))]))
    best_vals_reg.append(max(vals)) 
    
    
    vals_pca=[]
    for i in np.linspace(0.0001,0.005,100):
        predicted_probas = clf_pca.predict_proba(x_test_pca)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals_pca.append(metrics.auc(fpr, tpr))
    print('{0}-{1}'.format(max(vals_pca),np.linspace(0.0005,0.01,100)[vals_pca.index(max(vals_pca))]))
    best_vals_reg_pca.append(max(vals_pca)) 
    
    
    vals_over=[]
    for i in np.linspace(0.001,0.1,100):
        predicted_probas = clf_oversampling.predict_proba(x_test)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals_over.append(metrics.auc(fpr, tpr))
    print('{0}-{1}'.format(max(vals_over),np.linspace(0.001,0.1,100)[vals_over.index(max(vals_over))]))
    best_vals_reg_oversampling.append(max(vals_over)) 
    
    vals_over_pca=[]
    for i in np.linspace(0.001,0.1,100):
        predicted_probas = clf_oversampling_pca.predict_proba(x_test_pca)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals_over_pca.append(metrics.auc(fpr, tpr))
    print('{0}-{1}'.format(max(vals_over_pca),np.linspace(0.001,0.1,100)[vals_over_pca.index(max(vals_over_pca))]))
    best_vals_reg_oversampling_pca.append(max(vals_over_pca)) 
    
    print('-------------------------------------------------------------')




0.798357415485278-0.002611111111111111
0.8241662122864413-0.006833333333333334
0.696269538349691-0.035
0.6904761904761905-0.004
-------------------------------------------------------------
0.7096330275229358-0.000691919191919192
0.7111432604093155-0.006545454545454545
0.7046153846153846-0.019000000000000003
0.6958009880028229-0.037000000000000005
-------------------------------------------------------------
0.6701762782164792-0.0015555555555555555
0.7006859695301906-0.002994949494949495
0.7500731169072877-0.014000000000000002
0.7416979075270532-0.010000000000000002
-------------------------------------------------------------
0.7064046044575066-0.004242424242424242
0.755296350722508-0.00501010101010101
0.7796350722507961-0.031
0.79469752632868-0.029
-------------------------------------------------------------
0.7017792667038898-0.0013636363636363637
0.7298678578075563-0.002227272727272727
0.7448799553322166-0.019000000000000003
0.7630113530616043-0.012
-------------------------------

In [82]:
print(np.mean(best_vals_reg))
print(np.mean(best_vals_reg_pca))
print(np.mean(best_vals_reg_oversampling))
print(np.mean(best_vals_reg_oversampling_pca))

0.7016492012324222
0.7187310779151785
0.7261989655666798
0.7177092736836469


In [91]:
best_vals_reg=[]
for j in range(10):
    data_unique_car_number = pd.DataFrame(data['CAR NUMBER'].unique())
    data_unique_car_number.columns=['CAR NUMBER']
    
    data_random=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    data_random.columns=['RandomNum']
    
    data_unique_car_number['RandomNum']=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
    
    data_merged =pd.merge(data,data_unique_car_number,on='CAR NUMBER',how='left')

    data_new = data_merged.drop(['CAR NUMBER','LOCATION', 'Last Replacement Date','CUT OFF DATE','BUILT DATE',                                'CAR_AGE IN DAYS','LIFE TO DATE MILEAGE',
                                'Mileage since Last Replacement','COMPONENT_AGE IN DAYS'],axis =1)
    
    #train and test  
    data_train_temp = data_new[data_new['RandomNum'] <= 0.75]
    data_train = data_train_temp.drop(['RandomNum'],axis =1)
    
    data_test_temp = data_new[data_new['RandomNum'] > 0.75]
    data_test = data_test_temp.drop(['RandomNum'],axis =1)
    
        #train and test  
    data_train_temp = data_new[data_new['RandomNum'] <= 0.75]
    data_train = data_train_temp.drop(['RandomNum'],axis =1)
    
    data_test_temp = data_new[data_new['RandomNum'] > 0.75]
    data_test = data_test_temp.drop(['RandomNum'],axis =1)
    
    
    y_train = data_train['TARGET']
    x_train = data_train.drop(['TARGET'],axis =1)

    
    y_test = data_test['TARGET']
    x_test = data_test.drop(['TARGET'],axis =1)
    
    
    
    clf = RandomForestClassifier(random_state=1234,
                                    min_samples_split=2,
                                     max_leaf_nodes=3000,
                                     max_features=0.2,
                                     bootstrap = False, 
                                     n_estimators= 500,
                                     criterion='gini',
                                     min_samples_leaf=20).fit(x_train,y_train)
    
    
    vals=[]
    for i in np.linspace(0.01,0.2,50):
        predicted_probas = clf.predict_proba(x_test)
        y_predict_test = (predicted_probas [:,1] >= i).astype('int')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predict_test)
        vals.append(metrics.auc(fpr, tpr))
    print('{0}-{1}'.format(max(vals),np.linspace(0.01,0.2,50)[vals.index(max(vals))]))
    best_vals_reg.append(max(vals)) 

0.6026210698014512-0.013877551020408163
0.5614047097480833-0.07204081632653062
0.6509657434402333-0.013877551020408163
0.523059866962306-0.01
0.6905850091407678-0.01
0.75375-0.01
0.6747745375551313-0.01
0.6572915836719517-0.013877551020408163
0.6065817386308393-0.01
0.6275230793303083-0.025510204081632654


In [92]:
print(np.mean(best_vals_reg))

0.6348557338281072
